# import fcs data set

In [90]:
import flowio
import os
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_columns', 40)
used_cols = {   
    'SSC-W': 1,
    'SSC-A': 1,
    'SSC-H': 1,
    'FSC-W': 1,
    'FSC-A': 1,
    'FSC-H': 1,
    'Time': 0,
    'FJComp-BUV737-A': 1,
    'FJComp-APC-A': 1,
    'FJComp-BV711-A': 1,
    'FJComp-BB700-P-A': 1,
    'FJComp-BB630-A': 1,
    'FJComp-BUV395-A': 1,
    'FJComp-BUV563-A': 1,
    'FJComp-BV480-A': 1,
    'FJComp-BV421-A': 1,
    'FJComp-BV650-A': 1,
    'FJComp-BYG584-A': 1,
    'FJComp-PE-CF594-A': 1,
    'FJComp-BUV615-P-A': 1,
    'FJComp-BUV805-A': 1,
    'FJComp-BYG790-A': 1,
    'FJComp-PE-Cy5.5-A': 1,
    'FJComp-BV570-A': 1,
    'FJComp-BUV496-A': 1,
    'FJComp-BV605-A': 1,
    'FJComp-BB660-P-A': 0,
    'FJComp-BV786-A': 1,
    'FJComp-APC-R700-A': 1,
    'FJComp-BUV661-A': 0,
    'FJComp-BYG670-A': 1,
    'FJComp-BV750-P-A': 1,
    'FJComp-FITC-A': 1,
    'FJComp-APC-H7-A': 1,
    'FJComp-BB790-P-A': 0,
    'user_id': 0
}

In [91]:
# Directory containing FCS files
directory_path = './data/raw_fcs'
user_id_pattern = 'EU_\d+'
# Iterate through files in the directory
info = []
df_all = []
print(len(os.listdir(directory_path)))
for second_dir in os.listdir(directory_path):
    second_path = os.path.join(directory_path, second_dir)

    if os.path.isdir(second_path) and 'DS_Store' not in second_dir:
        file_name = os.listdir(second_path)[0]
    else:
        continue
    if file_name.endswith('.fcs'):
        file_path = os.path.join(second_path, file_name)
        try:
            fcs_file = flowio.FlowData(file_path)
            events = fcs_file.events
            
            npy_data = np.reshape(events, (-1, fcs_file.channel_count))
            
            column = [fcs_file.channels[str(i+1)]['PnN'] for i in range(fcs_file.channel_count)]
            user_id = re.search(user_id_pattern, second_dir).group()
            df = pd.DataFrame(npy_data, columns=column)
            if user_id:
                df['user_id'] = user_id
            else:
                df['user_id'] = -1
            info.append(df.describe())
            df_all.append(df)
        except Exception as e:
            print(f"Error loading {file_name}: {str(e)}")

df_all = pd.concat(df_all)

41


# look into the data
## basic concept
\# people:40
cell infomation in person:  
min \#: 680  
max \#: 363314  
mean: 126318  
25%:17350  
50%:90805  
75%:203955  
1. the range of the cell number is wide.
2. question: is the number matter? or is the percentage of the certain cell more important?
3. ans: both, and maybe the ratio looks like this A/A+B. or A+B/All
4. feature selection sould be more careful
## the data value
in each columns, the value represent the how strong the light it is  
1. some of the column sould using log
2. all value shoud apply minmax normalization

In [58]:
print(df_all[0].columns)

Index(['FSC-A', 'FSC-H', 'FSC-W', 'SSC-A', 'SSC-H', 'SSC-W', 'FJComp-APC-A',
       'FJComp-APC-H7-A', 'FJComp-APC-R700-A', 'FJComp-BB630-A',
       'FJComp-BB660-P-A', 'FJComp-BB700-P-A', 'FJComp-BB790-P-A',
       'FJComp-BUV395-A', 'FJComp-BUV496-A', 'FJComp-BUV563-A',
       'FJComp-BUV615-P-A', 'FJComp-BUV661-A', 'FJComp-BUV737-A',
       'FJComp-BUV805-A', 'FJComp-BV421-A', 'FJComp-BV480-A', 'FJComp-BV570-A',
       'FJComp-BV605-A', 'FJComp-BV650-A', 'FJComp-BV711-A',
       'FJComp-BV750-P-A', 'FJComp-BV786-A', 'FJComp-BYG584-A',
       'FJComp-BYG670-A', 'FJComp-BYG790-A', 'FJComp-FITC-A',
       'FJComp-PE-CF594-A', 'FJComp-PE-Cy5.5-A', 'Time', 'user_id'],
      dtype='object')


In [ ]:
for target_lasser in df_all[0].columns:
    if target_lasser == 'user_id':
        continue
    print(target_lasser)
    target = [i[target_lasser] for i in info]
    df_targer = pd.concat(target, axis=1)
    df_targer.columns = list(range(len(df_targer.columns)))
    df_targer = df_targer.T

    def display_info(col):
        display(df_targer[col].describe())
    display_cols = ['max', 'min', 'mean']

    display_info(display_cols)

# start to clean the data

In [92]:
from sklearn import preprocessing
import math

In [93]:

should_be_log = []
for i in df_all:
    if 'user' in i or 'Time' in i:
        continue
    des = df_all[i].describe()
    if des['min'] > 0: 
        value = math.log10(des['max']) - math.log10(des['min'])
    else:
        value = math.log10(des['max']) + math.log10(-des['min'])
    if value > 3:
        should_be_log.append(i)


In [94]:
df_all[should_be_log] = df_all[should_be_log].applymap(lambda x: math.log10(x) if x >= 1 else -math.log10(-x) if x <=-1 else 0)


In [95]:
drop_cols = [i for i in df_all if used_cols[i] == 0]
minmax_data = df_all.drop(columns=drop_cols)
input_dim = len(minmax_data.columns)
print(input_dim)

31


In [96]:

minmax = preprocessing.MinMaxScaler()
minmax_data = minmax.fit_transform(minmax_data)
minmax_permutation = np.random.permutation(minmax_data)
train_minmax = minmax_permutation[:int(len(minmax_data)*0.8)]
test_minmax = minmax_permutation[int(len(minmax_data)*0.8):]
minmax_data_var = np.var(train_minmax)
# minmax.data_max_
print(minmax_data_var)


0.04651723682738976


# grouping the data as cell infomation
using the vqvae code book as the grouping index. try different grouping number.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow as tf

2023-09-22 15:37:03.667406: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class VectorQuantizer(layers.Layer):
    def __init__(self, num_embeddings, embedding_dim, beta=0.25, **kwargs):
        super().__init__(**kwargs)
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings

        # The `beta` parameter is best kept between [0.25, 2] as per the paper.
        self.beta = beta

        # Initialize the embeddings which we will quantize.
        w_init = tf.random_uniform_initializer()
        self.embeddings = tf.Variable(
            initial_value=w_init(
                shape=(self.embedding_dim, self.num_embeddings), dtype="float32"
            ),
            trainable=True,
            name="embeddings_vqvae",
        )

    def call(self, x):
        # Calculate the input shape of the inputs and
        # then flatten the inputs keeping `embedding_dim` intact.
        input_shape = tf.shape(x)
        flattened = tf.reshape(x, [-1, self.embedding_dim])

        # Quantization.
        encoding_indices = self.get_code_indices(flattened)
        encodings = tf.one_hot(encoding_indices, self.num_embeddings)
        quantized = tf.matmul(encodings, self.embeddings, transpose_b=True)

        # Reshape the quantized values back to the original input shape
        quantized = tf.reshape(quantized, input_shape)

        # Calculate vector quantization loss and add that to the layer. You can learn more
        # about adding losses to different layers here:
        # https://keras.io/guides/making_new_layers_and_models_via_subclassing/. Check
        # the original paper to get a handle on the formulation of the loss function.
        commitment_loss = tf.reduce_mean((tf.stop_gradient(quantized) - x) ** 2)
        codebook_loss = tf.reduce_mean((quantized - tf.stop_gradient(x)) ** 2)
        self.add_loss(self.beta * commitment_loss + codebook_loss)

        # Straight-through estimator.
        quantized = x + tf.stop_gradient(quantized - x)
        return quantized

    def get_code_indices(self, flattened_inputs):
        # Calculate L2-normalized distance between the inputs and the codes.
        similarity = tf.matmul(flattened_inputs, self.embeddings)
        distances = (
            tf.reduce_sum(flattened_inputs ** 2, axis=1, keepdims=True)
            + tf.reduce_sum(self.embeddings ** 2, axis=0)
            - 2 * similarity
        )

        # Derive the indices for minimum distances.
        encoding_indices = tf.argmin(distances, axis=1)
        return encoding_indices

In [35]:
def get_encoder(input_dim, latent_dim=16):
    encoder_inputs = keras.Input(shape=(input_dim,))
    x = layers.Dense(128, activation='relu')(encoder_inputs)
    x = layers.Dense(64, activation='relu')(x)

    encoder_outputs = layers.Dense(latent_dim)(x)
    return keras.Model(encoder_inputs, encoder_outputs, name="encoder")


def get_decoder(output_dim, latent_dim=16):
    latent_inputs = keras.Input(shape=get_encoder(output_dim, latent_dim).output.shape[1:])
    x = layers.Dense(64, activation='relu')(latent_inputs)
    x = layers.Dense(128, activation='relu')(x)
    decoder_outputs = layers.Dense(output_dim)(x)
    return keras.Model(latent_inputs, decoder_outputs, name="decoder")

In [36]:
def get_vqvae(input_dim, latent_dim=16, num_embeddings=64):
    vq_layer = VectorQuantizer(num_embeddings, latent_dim, name="vector_quantizer")
    encoder = get_encoder(input_dim, latent_dim)
    decoder = get_decoder(input_dim, latent_dim)
    inputs = keras.Input(shape=(input_dim,))
    encoder_outputs = encoder(inputs)
    quantized_latents = vq_layer(encoder_outputs)
    reconstructions = decoder(quantized_latents)
    return keras.Model(inputs, reconstructions, name="vq_vae")


get_vqvae(31).summary()

Model: "vq_vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 31)]              0         
                                                                 
 encoder (Functional)        (None, 16)                13392     
                                                                 
 vector_quantizer (VectorQu  (None, 16)                1024      
 antizer)                                                        
                                                                 
 decoder (Functional)        (None, 31)                13407     
                                                                 
Total params: 27823 (108.68 KB)
Trainable params: 27823 (108.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
class VQVAETrainer(keras.models.Model):
    def __init__(self, train_variance, input_dim, latent_dim=32, num_embeddings=128, **kwargs):
        super().__init__(**kwargs)
        self.train_variance = train_variance
        self.latent_dim = latent_dim
        self.num_embeddings = num_embeddings

        self.vqvae = get_vqvae(input_dim, self.latent_dim, self.num_embeddings)

        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.vq_loss_tracker = keras.metrics.Mean(name="vq_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.vq_loss_tracker,
        ]

    def train_step(self, x):
        with tf.GradientTape() as tape:
            # Outputs from the VQ-VAE.
            reconstructions = self.vqvae(x)

            # Calculate the losses.
            reconstruction_loss = (
                tf.reduce_mean((x - reconstructions) ** 2) / self.train_variance
            )
            total_loss = reconstruction_loss + sum(self.vqvae.losses)

        # Backpropagation.
        grads = tape.gradient(total_loss, self.vqvae.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.vqvae.trainable_variables))

        # Loss tracking.
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.vq_loss_tracker.update_state(sum(self.vqvae.losses))

        # Log results.
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "vqvae_loss": self.vq_loss_tracker.result(),
        }

In [74]:
vqvae_trainer = VQVAETrainer(minmax_data_var, input_dim, latent_dim=24, num_embeddings=64)
vqvae_trainer.compile(optimizer=keras.optimizers.Adam())
vqvae_trainer.fit(minmax_data, epochs=20, batch_size=4096)

Epoch 1/20
1234/1234 [==============================] - 16s 12ms/step - loss: 6.2524 - reconstruction_loss: 0.5046 - vqvae_loss: 5.1669
Epoch 2/20
1234/1234 [==============================] - 14s 12ms/step - loss: 0.6526 - reconstruction_loss: 0.2432 - vqvae_loss: 0.3898
Epoch 3/20
1234/1234 [==============================] - 15s 12ms/step - loss: 0.6824 - reconstruction_loss: 0.2112 - vqvae_loss: 0.4698
Epoch 4/20
1234/1234 [==============================] - 15s 12ms/step - loss: 0.8168 - reconstruction_loss: 0.2103 - vqvae_loss: 0.6060
Epoch 5/20
1234/1234 [==============================] - 15s 12ms/step - loss: 0.8065 - reconstruction_loss: 0.2068 - vqvae_loss: 0.5988
Epoch 6/20
1234/1234 [==============================] - 15s 12ms/step - loss: 0.7671 - reconstruction_loss: 0.2071 - vqvae_loss: 0.5606
Epoch 7/20
1234/1234 [==============================] - 15s 12ms/step - loss: 0.7882 - reconstruction_loss: 0.2075 - vqvae_loss: 0.5801
Epoch 8/20
1234/1234 [==========================

In [75]:

trained_vqvae_model = vqvae_trainer.vqvae
idx = np.random.choice(len(test_minmax), 10)
reconstructions_test = trained_vqvae_model.predict(test_minmax[idx])
def difference(reconstruct, ground):
    for i in range(len(reconstruct)):
        print(tf.reduce_mean((ground[i] - reconstruct[i]) ** 2) / minmax_data_var)
        display(reconstruct[i] - ground[i])
        print('-------------------')
difference(reconstructions_test, test_minmax[idx])

1/1 [==============================] - 0s 183ms/step
tf.Tensor(0.8288087791973447, shape=(), dtype=float64)


array([ 0.07296502,  0.02280979,  0.0343242 ,  0.05841448,  0.05666576,
        0.03532025,  0.1964159 ,  0.07133903,  0.19771209,  0.23106134,
        0.14412222,  0.09105127,  0.45276904,  0.37528221,  0.22548456,
       -0.0366531 , -0.01320399, -0.03106298,  0.05971481,  0.21814827,
       -0.12833317,  0.00554435,  0.38559891, -0.09135276,  0.04316327,
        0.00863358,  0.19789152,  0.43170157,  0.20338359, -0.02806171,
        0.34877901])

-------------------
tf.Tensor(0.1811023079320415, shape=(), dtype=float64)


array([ 1.79384593e-01,  1.32812793e-01,  5.38383650e-02, -3.99978604e-02,
       -3.63099696e-02, -1.06013122e-02, -3.88241646e-02,  2.99879074e-02,
       -1.27910368e-04, -7.76650331e-02,  7.14738901e-03,  2.17432974e-01,
        3.03707445e-02,  8.19061781e-03, -4.00972613e-03,  1.18111933e-01,
       -7.35817205e-02, -6.52480002e-02, -4.18237320e-03,  2.98852439e-02,
       -2.29808087e-02,  1.92722409e-01, -1.43514805e-01, -5.78668094e-02,
       -5.52344612e-02, -5.37667771e-02, -1.21667319e-01,  1.79495054e-01,
        8.83952903e-02, -1.64837477e-02, -3.80762344e-02])

-------------------
tf.Tensor(0.24192150208687285, shape=(), dtype=float64)


...

-------------------
tf.Tensor(0.22195967253270318, shape=(), dtype=float64)


array([ 0.19004071,  0.16358486,  0.02013358, -0.08104482, -0.06363776,
       -0.05238323, -0.06050818, -0.16074469, -0.01523175, -0.02970979,
       -0.01149831, -0.07046002, -0.06902711,  0.19206313,  0.00942331,
       -0.12781534,  0.06749238, -0.03240734,  0.0024076 ,  0.00053159,
        0.22236615,  0.0025933 , -0.07235671, -0.1174359 , -0.10256966,
        0.03402063, -0.09708634,  0.2142326 , -0.08331954, -0.03847362,
       -0.0117967 ])

-------------------
tf.Tensor(0.27556630014996986, shape=(), dtype=float64)


array([ 0.11432953,  0.11463847, -0.00816755, -0.03665322, -0.01779703,
       -0.02831986, -0.06308534, -0.01941224, -0.01385842, -0.03888438,
       -0.02848687, -0.02838322, -0.05285966,  0.10116892, -0.02650351,
       -0.00965606,  0.14330994, -0.03708725, -0.00544462,  0.0122146 ,
        0.20655196, -0.09278547,  0.25124592, -0.02474644, -0.08187613,
       -0.06080631, -0.08068548, -0.05355689, -0.04894722,  0.24655534,
        0.35648194])

-------------------


# mapping the cell id back to original data

In [89]:
encoder = vqvae_trainer.vqvae.get_layer("encoder")
quantizer = vqvae_trainer.vqvae.get_layer("vector_quantizer")

encoded_outputs = encoder.predict(minmax_data)
codebook_indices = quantizer.get_code_indices(encoded_outputs)
codebook_indices = codebook_indices.numpy()


157898/157898 [==============================] - 212s 1ms/step


# feature engineering
cell number: 64  
cell ratio: 64

In [136]:
df_all['cell_id'] = codebook_indices
def counting_cell(df):
    idx, count = np.unique(df['cell_id'], return_counts=True)
    total_num = len(np.unique(codebook_indices))
    x = np.zeros(total_num)
    for i in range(len(idx)):
        x[idx[i]] = count[i]
    ratio_x = x/sum(x)
    x = np.concatenate([x, ratio_x])
    x_cols = [str(i) for i in range(total_num)] + ['r'+str(i) for i in range(total_num)]
    return pd.DataFrame([x], columns=x_cols)

df = df_all.groupby('user_id').apply(counting_cell)
id_idx = [i[0] for i in df.index]
df.index = id_idx


In [159]:
y = pd.read_csv('data/sick.csv')
y['file_flow_id'] = y['file_flow_id'].apply(lambda x: re.search(user_id_pattern, x).group())
y['label'] = y['label'].apply(lambda x: 1 if x == 'Sick' else 0)
y.index = y['file_flow_id'].to_numpy()
y = y.drop(columns='file_flow_id')
df = pd.concat([df, y], axis=1)

In [254]:
idx = np.random.permutation(len(df))
train = df.iloc[idx[:int(len(df)*0.8)]]
test = df.iloc[idx[int(len(df)*0.8):]]
test.iloc[:,:-1]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,r44,r45,r46,r47,r48,r49,r50,r51,r52,r53,r54,r55,r56,r57,r58,r59,r60,r61,r62,r63
EU_035,1216.0,2247.0,574.0,113.0,167.0,1407.0,494.0,744.0,2434.0,227.0,339.0,616.0,652.0,1259.0,3629.0,4341.0,948.0,980.0,2913.0,241.0,...,0.007681,0.009401,0.009255,0.048258,0.017811,0.004023,0.029690,0.015304,0.066011,0.004285,0.020114,0.002915,0.011835,0.000189,0.003994,0.018875,0.003848,0.009211,0.015916,0.005422
EU_047,1646.0,6143.0,6085.0,1322.0,1718.0,4973.0,250.0,4801.0,5916.0,958.0,2575.0,224.0,761.0,1766.0,15288.0,13726.0,11835.0,1604.0,10239.0,476.0,...,0.002206,0.023228,0.015342,0.016493,0.014451,0.010723,0.024844,0.023586,0.066910,0.001876,0.002991,0.005872,0.008513,0.000224,0.003683,0.023700,0.004147,0.004350,0.031233,0.015712
EU_021,782.0,523.0,819.0,2015.0,289.0,198.0,1635.0,3400.0,735.0,1155.0,273.0,1089.0,251.0,687.0,277.0,84.0,1319.0,309.0,123.0,1821.0,...,0.003018,0.010411,0.005225,0.013227,0.011870,0.000799,0.008382,0.012022,0.016232,0.003094,0.005694,0.009625,0.015091,0.008852,0.044702,0.005225,0.049166,0.011109,0.025540,0.017995
EU_014,280.0,245.0,176.0,57.0,26.0,137.0,38.0,293.0,210.0,21.0,25.0,29.0,176.0,146.0,789.0,1204.0,625.0,321.0,932.0,25.0,...,0.010385,0.009372,0.003040,0.032105,0.017920,0.004749,0.039007,0.015324,0.055344,0.002470,0.007409,0.000887,0.026976,0.000823,0.004053,0.012855,0.004496,0.020137,0.022416,0.008992
EU_022,120.0,296.0,2699.0,1751.0,669.0,318.0,90.0,817.0,329.0,505.0,1268.0,68.0,38.0,306.0,419.0,315.0,721.0,561.0,135.0,119.0,...,0.000474,0.045896,0.016905,0.012985,0.005423,0.008738,0.003087,0.007284,0.009996,0.000702,0.001241,0.016496,0.003430,0.001829,0.060465,0.004426,0.056300,0.002597,0.044230,0.019812
EU_043,498.0,1437.0,4909.0,3337.0,1693.0,647.0,93.0,964.0,1009.0,2268.0,1525.0,98.0,167.0,290.0,1196.0,870.0,808.0,124.0,470.0,152.0,...,0.003446,0.066143,0.008265,0.007735,0.015819,0.002361,0.011747,0.018481,0.019144,0.005349,0.005337,0.028927,0.007385,0.002373,0.019867,0.009289,0.025626,0.004000,0.095696,0.024819
EU_034,242.0,6835.0,2377.0,710.0,764.0,2056.0,15.0,1442.0,3036.0,846.0,1787.0,24.0,106.0,1511.0,4595.0,3094.0,1523.0,834.0,749.0,15.0,...,0.001450,0.024836,0.031945,0.035048,0.039388,0.003265,0.013954,0.035301,0.030799,0.000416,0.001572,0.008610,0.006288,0.000071,0.008539,0.020292,0.009208,0.002485,0.043151,0.007616
EU_031,4728.0,4891.0,3417.0,1528.0,967.0,1457.0,2903.0,2715.0,3460.0,1663.0,1021.0,2985.0,2982.0,1836.0,4033.0,2748.0,1186.0,1410.0,1670.0,1364.0,...,0.013842,0.017019,0.012637,0.027556,0.019339,0.002997,0.013391,0.016498,0.028316,0.005742,0.018192,0.005297,0.007701,0.003332,0.017464,0.009364,0.017013,0.043854,0.042990,0.014538


In [264]:
model = keras.Sequential([
    layers.Dense(32),
    layers.LeakyReLU(alpha=0.1),
    layers.BatchNormalization(fused=False),
    layers.Dense(16),
    layers.LeakyReLU(alpha=0.1),
    layers.BatchNormalization(fused=False),
    keras.layers.Dense(1, activation='sigmoid'),
])
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [267]:
history = model.fit(train.iloc[:,:-1], train['label'], epochs=15)

Epoch 1/15
1/1 [==============================] - 0s 15ms/step - loss: 0.5746 - accuracy: 0.7188
Epoch 2/15
1/1 [==============================] - 0s 13ms/step - loss: 0.5547 - accuracy: 0.7500
Epoch 3/15
1/1 [==============================] - 0s 7ms/step - loss: 0.5391 - accuracy: 0.7500
Epoch 4/15
1/1 [==============================] - 0s 17ms/step - loss: 0.5269 - accuracy: 0.7188
Epoch 5/15
1/1 [==============================] - 0s 5ms/step - loss: 0.5169 - accuracy: 0.7188
Epoch 6/15
1/1 [==============================] - 0s 12ms/step - loss: 0.5083 - accuracy: 0.7500
Epoch 7/15
1/1 [==============================] - 0s 30ms/step - loss: 0.5016 - accuracy: 0.7500
Epoch 8/15
1/1 [==============================] - 0s 6ms/step - loss: 0.4950 - accuracy: 0.7500
Epoch 9/15
1/1 [==============================] - 0s 7ms/step - loss: 0.4886 - accuracy: 0.7500
Epoch 10/15
1/1 [==============================] - 0s 5ms/step - loss: 0.4823 - accuracy: 0.7500
Epoch 11/15
1/1 [=================

In [268]:
model.evaluate(test.iloc[:,:-1], test['label'])

1/1 [==============================] - 0s 30ms/step - loss: 0.5335 - accuracy: 0.7500


[0.5334621667861938, 0.75]

# result
In this case, the number of the grouping should be consider as follow. First, How many kind of cell should be presented. Second, is it presenting well enough. There could be more method to estimate these two. 
   
Feature engineer can consider more piror distribution which means target important cell group by knowing domain knowledge. Eventually, I put all the features on, and choice nn model to learn.  
  
Then the classification model can do one more normalization but due to the lack of the instance, I decided not to do it here.  
  
Overall, the test accuracy flucuated between 0.375 to 0.75, mostly around 0.6 which is not to bad since the training accuaracy is around 0.7.